In [3]:
!pip install -q lxml
!pip install webdriver-manager

import bs4
import lxml
import pandas
import urllib

from urllib import request



In [4]:
url_ligue_1 = "https://fr.wikipedia.org/wiki/Championnat_de_France_de_football_2019-2020"
    
request_text = request.urlopen(url_ligue_1).read()
#print(request_text[:1000])
page = bs4.BeautifulSoup(request_text, "lxml")


In [5]:
print(page.find("title"))

<title>Championnat de France de football 2019-2020 — Wikipédia</title>


In [6]:
#trouver le tableau
# on identifie le tableau en question : c'est le premier qui a cette classe "wikitable sortable"
tableau_participants = page.find('table', {'class' : 'wikitable sortable'})
#print(tableau_participants)